In [56]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
from func import get_model2, get_model, get_data_loaders, get_patient_loader, MyNet, MyNet2
from torchvision import models
import os
import scipy

In [59]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
batch_size = 64
data_dir = "../Training_Data/"
criterion = torch.nn.CrossEntropyLoss()
#model = get_model()
model = MyNet(models.resnet50(weights="IMAGENET1K_V2"))
#model = MyNet(my_pretrained_model=models.resnet50(weights="IMAGENET1K_V2"))#path = "./data/05072024_single__5e-06resnet2.pt"
#path = "./data/09072024_lr__0.0005resnet.pt"
#model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

patients = [os.path.basename(f) for f in os.scandir(data_dir) if f.is_dir()]

In [60]:

    
# function that combines all tiles back to the image
def plot_data_mapped(data_dir, patient, category = "diff"):
    base_path = data_dir+patient+"/Preprocessed_STDataset/"
    merge = pd.read_csv(base_path + "merge.csv")
    merge["diff"] = merge["labels"] - merge["output"]
    merge.to_csv(base_path + "merge.csv")
    plt.scatter(merge['x'], merge['y'], c=merge[category], cmap='viridis')
    plt.colorbar(label='labels')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title(category + " for patient " + patient)
    plt.show()
    #merge.plot.scatter(x="x", y="y")
    
def plot_hist_comparison(data_dir, patient):
    base_path = data_dir+patient+"/Preprocessed_STDataset/"
    merge = pd.read_csv(base_path + "merge.csv")
    out = merge['output']
    plt.hist(out)
    plt.show()
    labels = merge['labels']
    plt.hist(labels)
    plt.show()
    
    
def print_metrics(data_dir, patient, metric):
    base_path = data_dir+patient+"/Preprocessed_STDataset/"
    merge = pd.read_csv(base_path + "merge.csv")
    out = merge['output']
    labels = merge['labels']
    mean_out = out.mean()
    std_out = out.std()
    mean_labels = labels.mean()
    std_labels = labels.std()
    if metric == "mean":
        print("mean diff: ", "{:.4f}".format(abs(mean_out-mean_labels)), ", mean out: ", "{:.4f}".format(mean_out), ", mean labels: ", "{:.4f}".format(mean_labels))
    if metric == "std":
        print("std diff: ", "{:.4f}".format(abs(std_out-std_labels)), ", std out: ", "{:.4f}".format(std_out), ", std labels: ", "{:.4f}".format(std_labels))

def plot_hist_comparison(data_dir, patient):
    base_path = data_dir+patient+"/Preprocessed_STDataset/"
    merge = pd.read_csv(base_path + "merge.csv")
    out = merge['output']
    plt.hist(out)
    plt.show()
    labels = merge['labels']
    plt.hist(labels)
    plt.show()
    
    
def print_metrics(data_dir, patient, metric):
    base_path = data_dir+patient+"/Preprocessed_STDataset/"
    merge = pd.read_csv(base_path + "merge.csv")
    out = merge['output']
    labels = merge['labels']
    mean_out = out.mean()
    std_out = out.std()
    mean_labels = labels.mean()
    std_labels = labels.std()
    if metric == "mean":
        print("mean diff: ", "{:.4f}".format(abs(mean_out-mean_labels)), ", mean out: ", "{:.4f}".format(mean_out), ", mean labels: ", "{:.4f}".format(mean_labels))
    if metric == "std":
        print("std diff: ", "{:.4f}".format(abs(std_out-std_labels)), ", std out: ", "{:.4f}".format(std_out), ", std labels: ", "{:.4f}".format(std_labels))
    if metric == "pearson":
        print("pearson corr: " + str(round(scipy.stats.pearsonr(out, labels)[0], 2)))
    
    
    
def plot_data_scatter(data_dir, patient, gene):
    base_path = data_dir+patient+"/Preprocessed_STDataset/"
    merge = pd.read_csv(base_path + "merge.csv")
    plt.scatter(merge['output'], merge['labels'])
    plt.text(x=-2, y=3 , s="pearson corr: " + str(round(scipy.stats.pearsonr(merge['output'], merge['labels'])[0], 2)))
    plt.plot( [-2,3],[-2,3], color='red' )
    plt.title("labels vs output for " + patient + " for gene " + gene)
    plt.xlabel('output')
    plt.ylabel('target')
    plt.show()
    
    
def process_and_plot_patient(model, device, criterion, data_dir, patient, gene = "RUBCNL", results_filename = "results.csv"):
    generate_results(model, device, criterion, data_dir, patient, gene, results_filename)
    merge_data(data_dir, patient, results_filename)
    plot_data_mapped(data_dir, patient, "labels")
    plot_data_mapped(data_dir, patient, "output")
    plot_data_mapped(data_dir, patient, "diff")
    plot_data_scatter(data_dir, patient, gene)
    print_metrics(data_dir, patient, "mean")
    print_metrics(data_dir, patient, "std")
    print_metrics(data_dir, patient, "pearson")
    
    print("----------------------------------------")

In [28]:
models_per_epoch_dir = "./RUBCNL_HLR_old_model/"

models_per_epoch = [os.path.basename(f) for f in os.scandir(models_per_epoch_dir)]
#models_per_epoch = [f for f in os.scandir(models_per_epoch_dir)]
m = models_per_epoch[0]

model = MyNet(models.resnet50(weights="IMAGENET1K_V2"))
model.load_state_dict(torch.load(models_per_epoch_dir + m, map_location=torch.device(device)))
process_and_plot_patient(model.to(device), device, criterion, data_dir, "p007", "RUBCNL", results_filename = "slim_model_results.csv")

In [29]:
process_and_plot_patient(old_model.to(device), device, criterion, data_dir, "p007", "RUBCNL", results_filename = "slim_model_results.csv")

In [61]:
import os
def get_img_target_name(loader, device, tile_no):
    image, target, name = loader[tile_no]
    image = image.unsqueeze(0).to(device)
    image = image.float()
    target = torch.tensor(target[0]).to(device)
    return image, target, name
patients = [os.path.basename(f) for f in os.scandir(data_dir) if f.is_dir()]
patients = patients[0:-1]#drop p008

patient = "/p009"
loader = get_patient_loader(data_dir, patient)

print(patients)
for i in range (20):
    
    input, target, name = get_img_target_name(loader,device,i)

In [32]:
def calc_residuals(test_df, true, preds):
  for i in range(len(true)):
    test_df[true[i]] = test_df[true[i]].astype(float)
    test_df[preds[i]] = test_df[preds[i]].astype(float)
    res_name = true[i].replace("true", "residuals")
    res_abs_name = true[i].replace("true", "residuals_abs")
    test_df[res_name] = test_df[true[i]] - test_df[preds[i]]
    test_df[res_abs_name] = test_df[res_name].abs()
  return test_df

In [33]:
import os
data_dir = "./data/"
patients = [os.path.basename(f) for f in os.scandir(data_dir) if f.is_dir()]
patients = patients[0:-1]
print(patients)
for patient in patients:
    if patient == 'p008':
        continue
    filename = data_dir + "/" + patient + "/Preprocessed_STDataset/spatial_data.csv"
    spatial_matrix = pd.read_csv(data_dir+patient+"/Preprocessed_STDataset/Raw_Spatial_Matrix_156_"+patient+".csv") #adapt to sample
    spatial_matrix['tile'] = spatial_matrix['tile'].apply(lambda x: "{}{}".format(x, ".tiff"))
    spatial_matrix['tile'] = spatial_matrix['tile'].apply(lambda x: "{}{}".format(data_dir+patient+"/Tiles_156/", x[57:])) #adapt to sample
    spatial_matrix['path'] = spatial_matrix['tile']
    spatial_matrix['tile'] = spatial_matrix['tile'].apply(lambda x: os.path.basename(x))
    spatial_matrix.to_csv(filename, index=False)

In [34]:

for patient in patients:
    merge_data(data_dir, patient, "results_slim_model.csv")

In [35]:

    
print(patients)
for patient in patients:
    
    if patient == 'p008':
        continue
    plot_data_mapped(data_dir, patient, "labels")
    plot_data_mapped(data_dir, patient, "output")
    plot_data_mapped(data_dir, patient, "diff")
    print("----------------------------------------")

In [36]:
# function that combines all tiles back to the image

    #merge.plot.scatter(x="x", y="y")
for patient in patients:
    
    
    if patient == 'p008':
        continue
    plot_data_scatter(data_dir, patient)

In [37]:

for patient in patients:
    
    if patient == 'p008':
        continue
    print_metrics(data_dir, patient, "mean")
for patient in patients:
    
    if patient == 'p008':
        continue
    print_metrics(data_dir, patient, "std")
    

In [38]:
from scipy import stats


In [39]:
for patient in patients:
    base_path = data_dir+patient+"/Preprocessed_STDataset/"
    merge = pd.read_csv(base_path + "merge.csv")
    corr = stats.pearsonr(merge['output'], merge['labels'])
    print(corr)

In [40]:
print(corr)

In [41]:
process_and_plot_patient(model, device, criterion, data_dir, patient, "RUBCNL", results_filename = "slim_model_results.csv")

In [42]:
old_model = get_model()
path = "./10082023_RUBCNL_ST_absolute_single_64_NLR_loss_resnet50.pt"
old_model = old_model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

In [ ]:

def get_res18(path):
    class Res18(nn.Module):
        def __init__(self, ciga):
            super(Res18, self).__init__()
            self.pretrained = ciga

            self.gene1 = nn.Sequential(nn.Linear(512, 200), nn.ReLU(), nn.Linear(200, 1))

        def forward(self, x):
            x = self.pretrained(x)
            x = self.gene1(x)
            return x

    ciga = models.resnet18()
    ciga.fc = torch.nn.Sequential()
    res18 = Res18(ciga)
    res18.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    return(res18)

In [ ]:
path = "./RUBCNL_HLR_old_model_now_all_layers/11072024_ep_29_lr_0.0005resnet.pt"
res18_no_ice = get_res18(path).to(device)
path = "./resnet_iced_e29.pt"
res18_ice    = get_res18(path)

In [ ]:
ms = [res18]

In [ ]:
#val 9 13
for patient in patients:
    if patient == "p008":
        continue
    process_and_plot_patient(res18_no_ice, device, criterion, data_dir, patient, "RUBCNL", results_filename = "slim_model_results.csv")

In [52]:

def get_res18(path):
    class Res18(nn.Module):
        def __init__(self, ciga):
            super(Res18, self).__init__()
            self.pretrained = ciga

            self.gene1 = nn.Sequential(nn.Linear(512, 200), nn.ReLU(), nn.Linear(200, 1))

        def forward(self, x):
            x = self.pretrained(x)
            x = self.gene1(x)
            return x

    ciga = models.resnet18()
    ciga.fc = torch.nn.Sequential()
    res18 = Res18(ciga)
    print(res18.load_state_dict(torch.load(path, map_location=torch.device('cpu'))))
    #return(res18)
def get_MyNet2(path):
    class MyNet2(nn.Module):
        def __init__(self, my_pretrained_model=models.resnet50(weights="IMAGENET1K_V2")):
            super(MyNet2, self).__init__()
            self.pretrained = my_pretrained_model
    
            self.gene1 = nn.Sequential(nn.Linear(1000, 200), nn.ReLU(), nn.Linear(200, 1))
    
        def forward(self, x):
            x = self.pretrained(x)
            gene1 = self.gene1(x)
            return gene1
    net = MyNet2()
    
    print(net.load_state_dict(torch.load(path, map_location=torch.device('cpu'))))
    
    return net
    

In [55]:
path = "./RUBCNL_HLR_old_model_now_all_layers/11072024_ep_29_lr_0.0005resnet.pt"
#res50 = get_M(path)
path = "./res18_not_iced_e29.pt"
res18_no_ice = get_res18(path).to(device)
path = "./res18_iced_e29.pt"
res18_ice    = get_res18(path)
#ms = [res50,res18_no_ice,res18_ice]

In [ ]:
import torch
import torch.nn as nn
class MyNet3(nn.Module):
    def __init__(self, my_pretrained_model):
        super(MyNet3, self).__init__()
        self.pretrained = my_pretrained_model


        self.my_new_layers = nn.Sequential(nn.Linear(1000, 200),
                                            nn.ReLU(),
                                            nn.BatchNorm1d(200),
                                            nn.Dropout(0.3),
                                            nn.Linear(200, 1))


    def forward(self, x):
        x = self.pretrained(x)
        output = self.my_new_layers(x)
        #output = nn.functional.log_softmax(x, dim=1) #adapt output function to prediction task
        return output

In [ ]:
import torchvision
old_model = MyNet3(torchvision.models.resnet50(weights="IMAGENET1K_V2"))
path = "../models/res50/10082023_RUBCNL_ST_absolute_single_64_NLR_loss_resnet50.pt"
old_model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

In [ ]:
print(old_model)

In [ ]:
for patient in patients:
    generate_results(old_model.to(device), device, torch.nn.CrossEntropyLoss(), data_dir, patient, "RUBCNL", "results_slim_model.csv")

In [ ]:
path = "../models/res18/RUBCNL_HLR_old_model_now_all_layers/11072024_ep_29_lr_0.0005resnet.pt"
res18_no_ice = get_res18(path).to(device)

In [43]:
import os
data_dir = "../Training_Data/"
#patients = [os.path.basename(f) for f in os.scandir(data_dir) if f.is_dir()]
patients = ["p009", "p013"]
for patient in patients:
    if patient == "p008":
        continue
    process_and_plot_patient(old_model.to(device), device, criterion, data_dir, patient, "RUBCNL", results_filename = "slim_model_results.csv")

In [44]:
models = ["../models/res50/RUBCNL_HLR_Res50slim/15072024_ep_39_lr_0.0005resnet.pt", "../models/res50/RUBCNL_HLR_Res50slim_ice/15072024_ep_39_lr_0.0005resnet.pt", "../models/res50/RUBCNL_HLR_Res50slim_optim_ice/RUBCNL_HLR_Res50slim_optim_ice15072024_ep_26_lr_0.0005resnet.pt"]

In [51]:
patients = ["p009", "p013"]
for patient in patients:
    if patient == "p008":
        continue
    for model_path in models:
        print(model_path)
        res50 = get_model2()
        print(res50.load_state_dict(torch.load(model_path, map_location=torch.device('mps'))))
        process_and_plot_patient(res50.to(device), device, criterion, data_dir, patient, "RUBCNL", results_filename = "slim_model_results.csv")